In [77]:
import pandas as pd

In [88]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [83]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df.DOlocationID.fillna(value=-1, inplace=True)
    df.PUlocationID.fillna(value=-1, inplace=True)
    
    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [84]:
df_train = read_dataframe('data/fhv_tripdata_2021-01.parquet')
df_val = read_dataframe('data/fhv_tripdata_2021-02.parquet')

In [85]:
categorical = ['PUlocationID', 'DOlocationID'] #['PU_DO'] 

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [86]:
len(dv.feature_names_)

525

In [87]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [92]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)

mean_squared_error(y_train, y_pred_train, squared=False)

10.5285191072072

In [93]:
y_pred_val = lr.predict(X_val)

mean_squared_error(y_val, y_pred_val, squared=False)

11.014283163400654